<a href="https://colab.research.google.com/github/shuuuuyu/114-1-Programing-Language/blob/main/hw_1_%E6%97%A5%E5%B8%B8%E6%94%AF%E5%87%BA%E9%80%9F%E7%AE%97%E8%88%87%E5%88%86%E6%94%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

功能:
目標：從 Sheet 讀「消費紀錄」→ 計總額/分類小計/AA 分攤 → 寫回 Sheet Summary 分頁。
AI 點子（可選）：請模型總結本週花錢習慣與建議（例如「外食過多」）。
Sheet 欄位：date, category, item, amount, payer


整體要求，我希望程式碼可以整齊，例如程式掛載在最前面，板塊一個一個寫，最後使用gradio顯示，我希望AI建議的功能先空著，我到時候再貼上，請你幫我留位置
目標:記帳軟體
板塊一:AA功能
我希望AA功能可以是輸入大家要支付的總金額，選單選擇加幾成服務費，顯示金額
接著最後有個按鈕，直接輸入到我的google sheet的外食欄位，喔對了，可以多加一格輸入，這餐吃的是甚麼
板塊二:記錄消費的時間、選擇種類(外食/買菜/日用品)、金額，項目名稱(例如外食就是食物名稱或餐廳名稱都可以，買菜就是材料名稱，日用品就是日用品名稱)
我希望在這裡加上連接外部google sheet的功能，我會把檔案設成知道連結的所有人都可以編輯，所以應該不需要資格憑證之類的，然後我會給你一個連結，這樣你應該可以用最簡單的邏輯去寫這段連接google sheet的程式
板塊三:AI點子，讀取近一周的數據，給予建議
我希望這一塊你可以先寫個按鈕，顯示最近一周的消費，分別顯示每個種類的消費明細
最後加個AI板塊，請模型總結本週花錢習慣與建議（例如「外食過多」）

In [29]:
# ==============================
# 掛載與套件載入
# ==============================
import pandas as pd
import gradio as gr
import datetime
import gspread
from google.colab import auth
from google.auth.transport.requests import Request
from google.auth import default

# ==============================
# Google 帳號授權
# ==============================
auth.authenticate_user()   # 會跳出 Google 登入
creds, _ = default()
gc = gspread.authorize(creds)

# 你的 Google Sheet ID
SHEET_ID = "1MbCem2-X-1vTViIxN9j5Eu_C6fh-AtZQkTaIrzOE7Ok"

# 打開第一個工作表
sh = gc.open_by_key(SHEET_ID)
worksheet = sh.sheet1


# ==============================
# 板塊一：AA 功能
# ==============================
def aa_function(total_amount, service_fee_percent, people_count, description):
    if service_fee_percent is None:
        service_fee_percent = 0
    if people_count is None or people_count <= 0:
        people_count = 1

    final_amount = total_amount * (1 + service_fee_percent / 100)
    per_person = final_amount / people_count

    now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    new_record = [now, "外食", final_amount, description, f"含{service_fee_percent}%服務費, {people_count}人分攤, 每人{per_person:.2f}"]

    worksheet.append_row(new_record)

    return f"總金額 {final_amount:.2f} 元，每人分攤 {per_person:.2f} 元，已新增到 Google Sheet。"


# ==============================
# 板塊二：記帳功能
# ==============================
def record_expense(expense_type, amount, item_name, note):
    now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    new_record = [now, expense_type, amount, item_name, note]

    worksheet.append_row(new_record)

    return f"成功記錄：{expense_type} - {item_name} - {amount} 元"


# ==============================
# 板塊三：AI 點子
# ==============================
def analyze_week():
    data = worksheet.get_all_records()
    df = pd.DataFrame(data)
    df["時間"] = pd.to_datetime(df["時間"], errors="coerce")

    one_week_ago = datetime.datetime.now() - datetime.timedelta(days=7)
    recent_df = df[df["時間"] >= one_week_ago]

    summary = recent_df.groupby("種類")["金額"].sum().reset_index()
    detail = recent_df[["時間", "種類", "金額", "項目名稱"]]

    return summary, detail


def ai_suggestion():
    return "AI 建議功能尚未實作，這裡會顯示模型輸出的建議。"


# ==============================
# Gradio 介面
# ==============================
with gr.Blocks() as demo:
    gr.Markdown("## 記帳軟體")

    # 板塊一
    with gr.Tab("AA 功能"):
        total_input = gr.Number(label="總金額")
        fee_input = gr.Dropdown([0, 5, 10], label="服務費 (%)", value=0)
        people_input = gr.Number(label="分攤人數", value=1)
        desc_input = gr.Textbox(label="餐點/描述")
        aa_btn = gr.Button("送出")
        aa_output = gr.Textbox(label="結果")
        aa_btn.click(fn=aa_function, inputs=[total_input, fee_input, people_input, desc_input], outputs=aa_output)

    # 板塊二
    with gr.Tab("記帳功能"):
        type_input = gr.Dropdown(["外食", "買菜", "日用品"], label="種類")
        amount_input = gr.Number(label="金額")
        item_input = gr.Textbox(label="項目名稱")
        note_input = gr.Textbox(label="備註")
        record_btn = gr.Button("記錄消費")
        record_output = gr.Textbox(label="結果")
        record_btn.click(fn=record_expense, inputs=[type_input, amount_input, item_input, note_input], outputs=record_output)

    # 板塊三
    with gr.Tab("AI 點子"):
        analyze_btn = gr.Button("查看最近一週消費")
        summary_output = gr.Dataframe(label="各種類總結")
        detail_output = gr.Dataframe(label="詳細消費")
        analyze_btn.click(fn=analyze_week, inputs=[], outputs=[summary_output, detail_output])

        suggestion_btn = gr.Button("AI 給建議")
        suggestion_output = gr.Textbox(label="AI 建議")
        suggestion_btn.click(fn=ai_suggestion, outputs=suggestion_output)

# 啟動
demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9894da82bdda2a6d9c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
